In [1]:
# import pandas_datareader.data as web
import pandas as pd
import numpy as np
import talib as ta


In [2]:
df = pd.read_csv('BA.csv')

df['Date'] = pd.to_datetime(df['Date'])

df.index = df['Date']

df.head()
# df.describe()


,Date,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,,
2015-02-03,2015-02-03,146.369995,148.000000,146.179993,147.339996,128.639938,5978200
2015-02-04,2015-02-04,145.970001,148.389999,145.929993,147.220001,128.535172,4766200
2015-02-05,2015-02-05,147.259995,148.869995,146.460007,148.600006,129.740036,4252600
2015-02-06,2015-02-06,148.619995,148.889999,147.699997,148.000000,129.216217,4398800
2015-02-09,2015-02-09,146.160004,148.160004,145.630005,147.580002,128.849487,4202900


In [3]:
df = df.drop(["Date"], axis=1)

In [4]:
#5.1. Step 1: extract technical indicators

#Define Technical Indicators
#_______________________________________________________________________

#Simple Moving Average  
sma = ta.SMA(df.Close,900)
df['MA25'] = sma
df = df.fillna(0)

# RSI
df["RSI"] = ta.RSI(df.Close,14)

# Stochastic K
df["Stoch_K"], df["Stoch_D"] = ta.STOCH(df["High"], df["Low"], df["Close"], fastk_period=14, slowk_period=3, slowk_matype=0, 
                                       slowd_period=3, slowd_matype=0)

# MACD
df['MACD'], df['MACDSignal'], df['MACDHist'] = ta.MACD(df.Close, fastperiod= 12, slowperiod= 26, signalperiod= 9)

# Williams R%
df["WILLR"] = ta.WILLR(df['High'], df['Low'], df['Close'], timeperiod=14) 

#df["WILLR"] = (WILLR - WILLR.min())/(WILLR.max() - WILLR.min())
df.tail()

,Open,High,Low,Close,Adj Close,Volume,MA25,RSI,Stoch_K,Stoch_D,MACD,MACDSignal,MACDHist,WILLR
Date,,,,,,,,,,,,,,
2020-01-27,316.209991,321.260010,314.579987,316.600006,316.600006,7468100,284.427944,39.800736,39.610963,32.293929,-6.286475,-5.699115,-0.587360,-66.530010
2020-01-28,318.049988,319.359985,314.000000,316.559998,316.559998,5502200,284.638578,39.775639,39.323804,37.478964,-6.322034,-5.823699,-0.498335,-64.521934
2020-01-29,324.350006,327.179993,319.140015,322.019989,322.019989,12454700,284.852078,44.883662,39.474169,39.469645,-5.842293,-5.827418,-0.014875,-50.525550
2020-01-30,320.500000,325.350006,317.640015,323.299988,323.299988,6022000,285.067200,46.039141,47.928693,42.242222,-5.297740,-5.721482,0.423742,-41.166436
2020-01-31,321.750000,321.929993,316.989990,318.269989,318.269989,4996800,285.279755,42.287379,51.987834,46.463565,-5.211977,-5.619581,0.407604,-52.344512


In [5]:
# 5.2. Step 2: trend analysis using technical indicators

#make MA25 into a series
MA25 = df.iloc[:,6]

# Up trend is 1, No trend is 0, and Down trend is -1
df['Trend'] = MA25.rolling('5d').apply(lambda x: np.sign(x[-1] - x[0]))

df.tail()
#If close > MA25 and MA25 is rising for last 5 days then 1 (Uptrend)
#If close < MA25 and MA25 is declining for last 5 days then -1 (Downtrend)
#If neither are true then 0 (Notrend)
def f(row):
    if row['Close'] > row['MA25'] and row['Trend'] == 1:
        val = 1
    elif row['Close'] < row['MA25'] and row['Trend'] == -1:
        val = -1
    else:
        val = 0
    return val
df['Trend'] = df.apply(f, axis=1)
# df.tail(n=1)

C:\Users\JMadd\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: Currently, 'apply' passes the values as ndarrays to the applied function. In the future, this will change to passing it as Series objects. You need to specify 'raw=True' to keep the current behaviour, and you can pass 'raw=False' to silence this warning
  import sys


In [6]:
#5.3. Step 3: trading signal generation from trend analysis

Close = df.iloc[:,3]
df["UpTrend"] = (Close - Close.rolling('3 D').min())/(Close.rolling('3 D').max() - Close.rolling('3 D').min()) * .5 + .5
df["DownTrend"] = (Close - Close.rolling('3 D').min())/(Close.rolling('3 D').max() - Close.rolling('3 D').min()) * .5

def g(row):
    if row['Trend'] == 1:
        val = row['UpTrend']
    elif row['Trend'] == -1:
        val = row['DownTrend']
    else:
        val = 0
    return val
df['Trade_Signal'] = df.apply(g, axis=1)
df.tail()

,Open,High,Low,Close,Adj Close,Volume,MA25,RSI,Stoch_K,Stoch_D,MACD,MACDSignal,MACDHist,WILLR,Trend,UpTrend,DownTrend,Trade_Signal
Date,,,,,,,,,,,,,,,,,,
2020-01-27,316.209991,321.260010,314.579987,316.600006,316.600006,7468100,284.427944,39.800736,39.610963,32.293929,-6.286475,-5.699115,-0.587360,-66.530010,1,NaN,NaN,NaN
2020-01-28,318.049988,319.359985,314.000000,316.559998,316.559998,5502200,284.638578,39.775639,39.323804,37.478964,-6.322034,-5.823699,-0.498335,-64.521934,1,0.5,0.0,0.5
2020-01-29,324.350006,327.179993,319.140015,322.019989,322.019989,12454700,284.852078,44.883662,39.474169,39.469645,-5.842293,-5.827418,-0.014875,-50.525550,1,1.0,0.5,1.0
2020-01-30,320.500000,325.350006,317.640015,323.299988,323.299988,6022000,285.067200,46.039141,47.928693,42.242222,-5.297740,-5.721482,0.423742,-41.166436,1,1.0,0.5,1.0
2020-01-31,321.750000,321.929993,316.989990,318.269989,318.269989,4996800,285.279755,42.287379,51.987834,46.463565,-5.211977,-5.619581,0.407604,-52.344512,1,0.5,0.0,0.5


In [7]:
#I added .tail(n=Length - 32) to omit values where the indicators = 0 since the machine learning agorithm trains on the values 
#of the indicators and the MACD doesn't generate values until 33 periods have passed.
Length = (df.shape[0] - 1100)


df2 = (df[['MA25', 'RSI', 'Stoch_K', 'Stoch_D', 'MACD', 'WILLR', 'Trade_Signal']].copy().tail(n=Length).copy())


#I get some na values without this and I'm worried that using this and
# having those values as zero will mess with the algorithms interpretation of the data
df2 = df2.fillna(0)

# df2.describe()
df2.head()

,MA25,RSI,Stoch_K,Stoch_D,MACD,WILLR,Trade_Signal
Date,,,,,,,
2019-06-18,246.623167,64.906244,82.218373,69.342184,-0.811875,-1.568902,1.000000
2019-06-19,246.867922,59.872392,88.992382,80.116578,0.427263,-22.101158,0.858342
2019-06-20,247.118633,63.445633,89.074074,86.761609,1.897389,-9.107719,1.000000
2019-06-21,247.365789,60.648081,84.278800,87.448419,2.785066,-15.954722,0.759493
2019-06-24,247.616511,61.926708,87.858293,87.070389,3.620311,-11.362681,0.000000


In [8]:
#5.5. Step 5: network structure creation and training using ELM

target_column = ['Trade_Signal'] 
predictors = list(set(list(df2.columns))-set(target_column))
#line 3 here fufills the data normalization of Step 5.4
df2[predictors] = (df2[predictors] - df2[predictors].min())/(df2[predictors].max() - df2[predictors].min())

#df2.describe()
#df2.info()
df2.tail()
# df2.head()

,MA25,RSI,Stoch_K,Stoch_D,MACD,WILLR,Trade_Signal
Date,,,,,,,
2020-01-27,0.977965,0.338411,0.381134,0.284823,0.122619,0.336545,0.0
2020-01-28,0.983413,0.337842,0.377928,0.346214,0.120684,0.356737,0.5
2020-01-29,0.988936,0.453627,0.379607,0.369784,0.146792,0.497472,1.0
2020-01-30,0.994501,0.479819,0.473999,0.402612,0.176427,0.591579,1.0
2020-01-31,1.000000,0.394776,0.519317,0.452594,0.181094,0.479182,0.5


In [9]:
X = df2[predictors]

y = df2[target_column]

y = y.T

print(X.shape); print(X.ndim); print(y.shape); print(y.ndim)
# print(X)

(158, 6)
2
(1, 158)
2


In [10]:
Xindex = np.array([[1], [2], [3], [4], [5], [6]])

Xindex = Xindex.T

print(Xindex.shape); print(Xindex)

(1, 6)
[[1 2 3 4 5 6]]


In [11]:
#  Functional Expansion Block

# import sys
# np.set_printoptions(threshold=sys.maxsize)

feb = np.outer(X, Xindex)
# .flatten(order='F')
print(feb.shape); print(feb);
# print(feb.type)

(948, 6)
[[0.98973749 1.97947498 2.96921248 3.95894997 4.94868746 5.93842495]
 [0.         0.         0.         0.         0.         0.        ]
 [0.7234805  1.446961   2.1704415  2.893922   3.6174025  4.340883  ]
 ...
 [0.39477638 0.78955276 1.18432915 1.57910553 1.97388191 2.36865829]
 [0.18109449 0.36218898 0.54328347 0.72437795 0.90547244 1.08656693]
 [0.51931736 1.03863473 1.55795209 2.07726945 2.59658682 3.11590418]]


In [12]:
feb = feb.reshape(-1, 6)
print(feb)

[[0.98973749 1.97947498 2.96921248 3.95894997 4.94868746 5.93842495]
 [0.         0.         0.         0.         0.         0.        ]
 [0.7234805  1.446961   2.1704415  2.893922   3.6174025  4.340883  ]
 ...
 [0.39477638 0.78955276 1.18432915 1.57910553 1.97388191 2.36865829]
 [0.18109449 0.36218898 0.54328347 0.72437795 0.90547244 1.08656693]
 [0.51931736 1.03863473 1.55795209 2.07726945 2.59658682 3.11590418]]


In [13]:
feb =np.sum(feb, axis=1)
print(feb.shape); print(feb); 

(948,)
[2.07844873e+01 0.00000000e+00 1.51930905e+01 1.90571861e+01
 8.83163750e+00 1.79934391e+01 1.64489428e+01 1.32962240e-01
 1.78720754e+01 1.66610052e+01 1.02477854e+01 1.95816558e+01
 1.91926080e+01 2.69159807e-01 1.95243204e+01 1.83619156e+01
 1.19279172e+01 1.96008091e+01 1.77468104e+01 4.03425837e-01
 1.96950912e+01 1.70302438e+01 1.29423985e+01 1.84765217e+01
 1.87164553e+01 5.39629437e-01 1.96010965e+01 1.76388872e+01
 1.38969570e+01 1.93157603e+01 1.53380047e+01 6.72827092e-01
 1.89960095e+01 1.55566816e+01 1.41719294e+01 1.78891168e+01
 1.85883610e+01 8.10153407e-01 1.88984659e+01 1.72512619e+01
 1.48494400e+01 1.82005867e+01 1.22727090e+01 9.41178069e-01
 1.76611743e+01 1.29082573e+01 1.43266982e+01 1.58156646e+01
 1.22669376e+01 1.07310212e+00 1.65263976e+01 1.29046219e+01
 1.38641849e+01 1.46790191e+01 7.90034090e+00 1.19908667e+00
 1.38831357e+01 1.02595518e+01 1.27667238e+01 1.07232310e+01
 6.03307823e+00 1.32415375e+00 1.12665796e+01 9.51081334e+00
 1.16629351e+01 8

In [14]:
# feb = np.tanh(feb)
# print(feb)

In [15]:
#  Functional Expansion Block

# feb = np.outer(X, Xindex).flatten(order='F')
# feb = np.sum(cx.reshape(-1, 6), axis=1)
# feb = np.tanh(feb)

#all on one line
feb = np.tanh(np.sum(np.outer(X, Xindex).flatten(order='F').reshape(-1, 6), axis=1))

print(feb.shape); print(feb.ndim);
# print(feb)

(948,)
1


In [16]:
cx = feb.reshape(1, len(feb) )
print(cx.shape); print(cx.ndim);
# print(cx)

(1, 948)
2


In [17]:
# initialize random weights
Y = np.random.random((1, (Length * 6)))

print(Y.shape); print(Y.ndim);print(Y)

(1, 948)
2
[[0.13666795 0.50815067 0.40911807 0.19191172 0.62224849 0.16681504
  0.4789347  0.2648911  0.85550229 0.88315371 0.58344456 0.42686893
  0.03761763 0.77796701 0.61522738 0.59000188 0.62714218 0.6186661
  0.38565283 0.7155241  0.31660401 0.81029521 0.3414141  0.69791005
  0.24268663 0.8869122  0.50829489 0.40884326 0.23788122 0.84063106
  0.41984257 0.20215506 0.57326873 0.18717747 0.03321512 0.42409077
  0.14146825 0.88585301 0.51999232 0.24338729 0.99355173 0.21014053
  0.09501949 0.22326769 0.03219859 0.08868163 0.45351445 0.67586627
  0.60474522 0.51440012 0.57428206 0.08446444 0.76865383 0.14445906
  0.61293431 0.13932151 0.44055106 0.84913819 0.28011912 0.88489929
  0.6890582  0.9369954  0.4804955  0.23989444 0.59428142 0.43541304
  0.19813209 0.28042506 0.82094064 0.51130982 0.83054727 0.38637947
  0.7976039  0.60208005 0.38586796 0.22022705 0.32811617 0.63274761
  0.7241723  0.03792257 0.75202922 0.49680258 0.49293794 0.16957709
  0.79237369 0.78148727 0.484616   0.3

In [18]:
I = np.identity(len(feb))

print(I.shape); print(I.ndim); print(I)

(948, 948)
2
[[1. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 1.]]


In [19]:
# W= (cx.T).dot(cx)

# lamb = 1
# W = np.linalg.lstsq(cx.T.dot(cx) + lamb * I, cx.T.dot(Y),  rcond=1)

# W = ((cx.T.dot(cx) + lamb * I)**-1) * (cx.T.dot(Y))



lamb = 1
W = np.linalg.lstsq(cx.T.dot(cx) + lamb * I, cx.T.dot(Y))[0]  

# 
print(W.shape);  print(W.ndim); print(W)


C:\Users\JMadd\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  # This is added back by InteractiveShellApp.init_path()


(948, 948)
2
[[0.0001463  0.00054396 0.00043795 ... 0.00062472 0.00078748 0.00052664]
 [0.00014629 0.00054392 0.00043791 ... 0.00062467 0.00078742 0.0005266 ]
 [0.00014636 0.00054418 0.00043812 ... 0.00062497 0.0007878  0.00052685]
 ...
 [0.00014642 0.00054441 0.00043831 ... 0.00062523 0.00078813 0.00052707]
 [0.00014642 0.00054441 0.00043831 ... 0.00062523 0.00078813 0.00052707]
 [0.00014642 0.00054441 0.00043831 ... 0.00062523 0.00078813 0.00052707]]


In [20]:
cx_index = np.array(len(feb))
print(cx.shape);  print(cx.ndim);
# print(cx)

(1, 948)
2


In [21]:

y = np.outer(W, cx_index).reshape(948, -1)
# print(y)

In [38]:
# y= np.outer(W, cx_index).flatten(order='F')
# print(y.shape)

In [22]:
y = np.sum(np.outer(W, cx).reshape(948, -1), axis=1)
# print(y.shape)

In [23]:
print(y.shape)

(948,)


In [24]:
# import sys
# np.set_printoptions(threshold=sys.maxsize)

print(y)

[467.99078329 467.95571521 468.17846037 468.12342331 468.19350057
 468.01782556 468.17315493 467.55636415 467.32521054 464.17681068
 458.97535728 450.47458725 446.26694079 410.00190436 414.72660806
 395.74225992 440.52776853 459.62308201 460.64918377 463.51294296
 467.63513712 466.7270107  468.15286266 467.96912397 468.04746816
 466.49009121 455.01344549 425.76160472 352.88525569 396.84509215
 351.88657187 308.41334285 343.66633558 282.73116017 295.38984423
 289.32873178 353.81509303 385.32804709 374.94774801 394.48621648
 274.35869256 377.36716968 401.83319645 440.39320402 450.39710312
 465.75849061 467.76839153 467.93284185 468.10198745 467.95496668
 468.11470539 468.19128704 468.25624977 468.10705365 468.12379237
 468.14374277 468.20830572 468.13255599 468.30062554 468.33731408
 468.30785417 468.32800887 468.31727839 468.34371907 468.352303
 468.34941472 468.3236455  468.26918754 468.28469969 468.30759764
 468.32556844 468.2964571  468.18869425 466.51080975 462.39563494
 461.5469508

In [ ]:
Outputnp.mean(a, axis=1)

In [ ]:
#I'm working on figure 1 of the article and I have the expanded inputs CX, and the initialized weights as z
#I don't know how to read equation 2 or the Moore-Penros generalized inverse function of equation 5. If I'm reading this correctly
#(I'm not 100% sure I am), those functions are the key to using my CX expanded output values to create W.

# I'm still new to ELM so if I'm missing something please let me know. 
#any help is appreciated
#Thank you so much !!!!

In [ ]:
#this is a for loop  I found online. It is not what is described in the article

for iteration in range(1000):
    z = np.dot(cx.T, Y)
    sigmoid = 1/(1+np.exp(-cx))
    error = (y - sigmoid)
print(error); print(error.shape)

In [ ]:
sigmoidDerivative = sigmoid * (1 - sigmoid)
print(sigmoidDerivative)

In [ ]:
synaptic_weights += np.dot(cx, error*sigmoidDerivative)
print(synaptic_weights)